In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import tensorflow as tf

np.random.seed(778)
tf.random.set_seed(778)

k=3
class Individual:

    def __init__(self, num_of_features, num_of_classes, n=k, model=None, vector=None):
        self.fitness = None
        self.set_model(model, num_of_features, num_of_classes, n)
        if vector is not None:
            self.vector = vector
            self.decode()
        else:
            self.encode()
       
    def set_model(self, model, num_of_features, num_of_classes, n):
        
        if model is None:
            # Create a sequential model with n Dense layers
            model = tf.keras.Sequential([
                tf.keras.layers.Dense(num_of_features * 2 + 1, activation='relu', input_shape=(num_of_features,)),
            ])
            for _ in range(n - 1):
                model.add(tf.keras.layers.Dense(units=(num_of_features + n) // 2, activation='relu'))

            if num_of_classes == 2:
                model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
            else:
                model.add(tf.keras.layers.Dense(num_of_classes, activation='softmax'))

        self.model = model
    
    def encode(self):#Turning the model to the vector
        self.vector = np.concatenate([w.ravel() for w in self.model.get_weights()])
        self.decode()
    def decode(self):#Turnng the vector to the model
        weights_and_biases = []
        idx = 0
        for layer in self.model.layers:
            weight_shape = layer.get_weights()[0].shape
            bias_shape = layer.get_weights()[1].shape
            weight_size = np.prod(weight_shape)
            bias_size = np.prod(bias_shape)
            weights_and_biases.extend([self.vector[idx:idx + weight_size].reshape(weight_shape),
                                       self.vector[idx + weight_size:idx + weight_size + bias_size].reshape(bias_shape)])
            idx += weight_size + bias_size
        self.model.set_weights(weights_and_biases)
 
    def evaluate(self, x, y, batch_size=16): #Objective Function
        
        if self.fitness is not None: #trick to make the function faster
            return self.fitness

        predictions = self.model.predict(x, batch_size=batch_size)
        output_activation = self.model.layers[-1].activation
        
        #changing the way of calculate the predicitions based on last layer(num of classes 2 or more)
        if output_activation == tf.keras.activations.softmax:
            predictions = tf.argmax(predictions, axis=1)
        elif output_activation == tf.keras.activations.sigmoid:
            predictions = tf.cast(tf.round(predictions), tf.int32)

        num_correct = np.sum(predictions == y)
        p = len(y)
        self.fitness = (100 / p) * num_correct
        return self.fitness


In [ ]:
class Population:
    def __init__(self,num_of_features,num_of_classes,X,y,iter=300, np=50, jr=0.3, cr=0.9, n=3,f=0.5):
        self.iter = iter
        self.np = np
        self.jr = jr
        self.cr = cr
        self.n = n
        self.f = f
        self.num_of_features=num_of_features
        self.num_of_classes = num_of_classes
        self.X = X
        self.y = y
        self.max_history = []
        self.avg_history = []
        self.centroid_history=[]
        self.x_best_idx = 0
        self.patience = 0
        self.x_best_prev = None
        self.initial_mutation_rate = 0.5
    #MUTATION
    def mutation(self, off_spring, target, x_best): #vi = xi +F*(xbest xi)+F*(xr1-xr2)
        #get the 2 random vector based on the equation 
        r1, r2 = np.random.choice(self.np, size=2, replace=False)
        
        xr1 = off_spring[r1]
        xr2 = off_spring[r2]
        
        # Self adaptive mutation based on this equation 
        #(initial_mutation_rate× (best_fitness−worst_fitness)) / (best_fitness−average_fitness)
​       best_fitness = off_spring[self.x_best_idx].evaluate(self.X, self.y)
        average_fitness = np.mean([ind.evaluate(self.X, self.y) for ind in off_spring])
        worst_fitness = min([ind.evaluate(self.X, self.y) for ind in off_spring])
        self.mutation_rate = self.initial_mutation_rate * ((best_fitness - average_fitness) / (best_fitness - worst_fitness))

        mutated_vector = np.array(target.vector + self.f * (x_best.vector - target.vector) + self.f * (xr1.vector - xr2.vector))
        mutated_vector += self.mutation_rate * np.random.normal(size=len(mutated_vector))  # Apply mutation rate
        mutated = Individual(num_of_features=self.num_of_features, num_of_classes=self.num_of_classes, vector=mutated_vector)
        #vi = mutated
        return mutated

    #SELECTION
    def selection(self, off_spring):#selection has two ways OBL or Centroid
        if np.random.random() < self.jr: #OBL Path
            vectors = np.array([individual.vector for individual in off_spring])
            l = np.min(vectors, axis=0)
            u = np.max(vectors, axis=0)
            off_spring = self.OBL(len(vectors[0]), off_spring, l, u, off_spring[0].model)
        else: #Centroid Path
            #Centroid is average summtion of best N individuals
            sorted_indices = np.argsort([individual.evaluate(self.X, self.y) for individual in off_spring])
            sorted_offspring = [-off_spring[i] for i in sorted_indices[:self.np]]
            centroid = np.mean([individual.vector for individual in sorted_offspring[:self.n]], axis=0)
            centroid = Individual(self.num_of_features, self.num_of_classes, vector=centroid)
            off_spring[-1] = centroid
            self.centroid_history.append(centroid.evaluate(self.X, self.y))
        return off_spring

    #CROSS_OVER
    def cross_over(self, target_vector, mutated_vector):
               #vij rand(01) CR or j ==jrand (Mutated Vector)
        #uij = 
               # xij otherwise (Target Vector)
        d = len(target_vector.vector)
        jrand = np.random.randint(0, d)
        ui = np.zeros(d)
        for j in range(d):
            if np.random.rand() < self.cr or jrand == j:
                ui[j] = mutated_vector.vector[j]
            else:
                ui[j] = target_vector.vector[j]
        ui = Individual(self.num_of_features,self.num_of_classes,vector=ui)
        
        return ui


    def start(self):
      #dfine the variables for the populotion   
      off_spring = [Individual(num_of_features=self.num_of_features,num_of_classes=self.num_of_classes) for _ in range(self.np)]
      d=len(off_spring[0].vector) # num of dim
      l = np.full(d,-3.0) #lower vector which used in obl function
      u = np.full(d,3.0)  #upper vector which used in obl function
        
      off_spring = self.OBL(d,off_spring,l,u,off_spring[0].model)
    
      fitnesses = [off_spring[i].evaluate(self.X,self.y) for i in range(self.np)]
      self.x_best_idx = np.argmax(fitnesses)
      self.max_history.append(fitnesses[self.x_best_idx])
      self.avg_history.append(np.mean(fitnesses))
      
      #index to itrate over the pop to do cross over on each one of them  
      idx = 0
      #target list which is represnt the order of indvidiual in pop which will do cross over on in random way
      target_list = np.random.permutation(self.np-1)


      for nfe in range(self.iter):
        # if we itrate over all invidiuals to do crossover reset everything to itrate again
        if idx == self.np-1:
          target_list = np.random.permutation(self.np-1)
          idx=0
        
        #Extinction Code
        #if the best individual not increase for 40 Consecutive generations do Extinction
        if np.abs(fitnesses[self.x_best_idx] - self.max_history[-1]) < 0.1:
            self.patience += 1
            if self.patience == 40:
                print("Reached patience limit. Resetting population.")
                self.patience = 0
                #replace the half of the individuals and don't pick the centroid or the best
                idxs = np.random.choice(self.np-1, self.np//2, replace=False)
                for i in idxs:
                    if i == self.x_best_idx:
                        continue
                    off_spring[i] = Individual(num_of_features=self.num_of_features, num_of_classes=self.num_of_classes)
        else:#resent the patience
            self.patience = 0  
        
        
        target_vector = off_spring[target_list[idx%self.np-1]]
        fitness = [individual.evaluate(self.X,self.y) for individual in off_spring]
        self.x_best_idx = np.argmax(fitness)
        x_best = off_spring[self.x_best_idx]
        #mutation call
        mutated_vector = self.mutation(off_spring,target_vector,x_best)
        #crossover call
        ui = self.cross_over(target_vector,mutated_vector)
 
        if ui.evaluate(self.X,self.y) > target_vector.evaluate(self.X,self.y):
          off_spring[target_list[idx]] = ui

        idx += 1

        
        off_spring = self.selection(off_spring)
        fitnesses = [off_spring[i].evaluate(self.X,self.y) for i in range(self.np)]
        self.max_history.append(np.max(fitnesses))
        self.avg_history.append(np.mean(fitnesses))
      self.x_best_idx = np.argmax(fitness)
      return off_spring[self.x_best_idx]




     
    def OBL(self, d, Pop, L, U,model):

        OP_op = np.zeros((self.np, d))
        #applay the quasi-opposite equation
        # xi = rand.uniform[ (ai +bi)/2,(ai+bi-xi)] i is index i in range of dim
        for i in range(self.np):
            for j in range(d):
                OP_op[i][j] = np.random.uniform((L[j] + U[j]) / 2, L[j] + U[j] - Pop[i].vector[j])

        fitness_values = []
        opposite_individual = []
        
        for individual_vector in OP_op:
            individual = Individual(num_of_features=self.num_of_features, num_of_classes=self.num_of_classes,vector=individual_vector)
            opposite_individual.append(individual)



        #create new pop of best np individuals from POP union opposite pop
        new_population = [Pop[i] if Pop[i].evaluate(self.X, self.y) > opposite_individual[i].evaluate(self.X, self.y)
                       else opposite_individual[i] for i in range(self.np)]


        #change one index to the obl to not make the pop stuck
        random_idx  = np.random.randint(self.np-1)
        if random_idx == self.x_best_idx:
            random_idx = (random_idx + 1) % self.np-1
        new_population[random_idx] = opposite_individual[random_idx]
        fitness_values = [i.evaluate(self.X,self.y) for i in new_population]
        print(fitness_values)
        return new_population

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
names = ['id', 'clump_thickness', 'uniformity_cell_size', 'uniformity_cell_shape', 'marginal_adhesion',
         'single_epithelial_size', 'bare_nuclei', 'bland_chromatin', 'normal_nucleoli', 'mitoses', 'class']
df = pd.read_csv(url, names=names)

# Preprocessing
# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

# Convert bare_nuclei column to numeric
df['bare_nuclei'] = pd.to_numeric(df['bare_nuclei'])

# Impute missing values with median
imputer = SimpleImputer(strategy='median')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df['class'] = df['class'].map({2: 0, 4: 1})
# Split features and target
X = df.drop(['id', 'class'], axis=1)
y = df['class']
y = np.array(y)
y = y.reshape(-1,1)
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
num_of_features = X_train.shape[1]
num_of_classes = len(np.unique(y_train))
pop = Population(num_of_features,num_of_classes,X,y)
best_nn = pop.start()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from google.colab import files
generations = range(1, len(pop.max_history) + 1)
fitness_values = pop.max_history

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5, 3))
ax.set_xlabel('Generation')
ax.set_ylabel('Fitness')
ax.set_title('MAX Fitness CenDE_OBL V2 (cancer)')

# Function to update the plot for each frame
def update_plot(frame):
    ax.clear()
    ax.plot(generations[:frame], fitness_values[:frame], marker='o')
    ax.set_xlabel('Generation')
    ax.set_ylabel('Fitness')
    ax.set_title(f'MAX Fitness CenDE_OBL V2 (cancer) (Generation {frame})')
    ax.grid(True)

# Create the animation
ani = animation.FuncAnimation(fig, update_plot, frames=len(generations), interval=200, repeat=False)

# Display the animation
HTML(ani.to_jshtml())


In [ ]:
best_nn.fitness

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


generations = range(1, len(pop.avg_history) + 1)
fitness_values = pop.avg_history

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5, 3))
ax.set_xlabel('Generation')
ax.set_ylabel('Fitness')
ax.set_title('AVG Fitness CenDE_OBL V2 (cancer)')

# Function to update the plot for each frame
def update_plot(frame):
    ax.clear()
    ax.plot(generations[:frame], fitness_values[:frame], marker='o')
    ax.set_xlabel('Generation')
    ax.set_ylabel('Fitness')
    ax.set_title(f'Fitness Progress (Generation {frame})')
    ax.grid(True)

# Create the animation
ani = animation.FuncAnimation(fig, update_plot, frames=len(generations), interval=200, repeat=False)

# Display the animation
HTML(ani.to_jshtml())



In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Encode the target labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)




# Create an instance of the Individual class
num_of_features = X_train.shape[1]
num_of_classes = len(np.unique(y_train))
individual = Individual(num_of_features, num_of_classes)

# Train the model (this will also decode the weights)
individual.decode()

# Evaluate the model on the test set
num_correct = individual.evaluate(X_test, y_test)
print("Fitness:", num_correct)


In [ ]:
num_of_features = X_train.shape[1]
num_of_classes = len(np.unique(y_train))
pop = Population(num_of_features,num_of_classes,X,y,iter=85)
best_nn = pop.start()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

generations = range(1, len(pop.max_history) + 1)
fitness_values = pop.max_history

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5, 3))
ax.set_xlabel('Generation')
ax.set_ylabel('Fitness')
ax.set_title('MAX Fitness CenDE_OBL V2 (iris)')

# Function to update the plot for each frame
def update_plot(frame):
    ax.clear()
    ax.plot(generations[:frame], fitness_values[:frame], marker='o')
    ax.set_xlabel('Generation')
    ax.set_ylabel('Fitness')
    ax.set_title(f'MAX Fitness CenDE_OBL V2 (iris) (Generation {frame})')
    ax.grid(True)

# Create the animation
ani = animation.FuncAnimation(fig, update_plot, frames=len(generations), interval=200, repeat=False)

# Display the animation
HTML(ani.to_jshtml())

video_filename = 'MAX_Fitness_CenDE_V2_OBL_(iris).mp4'

# Save the animation as a video using ffmpeg writer
ani.save(video_filename, writer='ffmpeg', fps=10)

# Download the video file
files.download(video_filename)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


generations = range(1, len(pop.avg_history) + 1)
fitness_values = pop.avg_history

# Create a figure and axis
fig, ax = plt.subplots(figsize=(5, 3))
ax.set_xlabel('Generation')
ax.set_ylabel('Fitness')
ax.set_title('AVG Fitness CenDE_OBL V2 (cancer)')

# Function to update the plot for each frame
def update_plot(frame):
    ax.clear()
    ax.plot(generations[:frame], fitness_values[:frame], marker='o')
    ax.set_xlabel('Generation')
    ax.set_ylabel('Fitness')
    ax.set_title(f'Fitness Progress (Generation {frame})')
    ax.grid(True)

# Create the animation
ani = animation.FuncAnimation(fig, update_plot, frames=len(generations), interval=200, repeat=False)

# Display the animation
HTML(ani.to_jshtml())

video_filename = 'AVG_Fitness_CenDE_V2_OBL_(iris).mp4'

# Save the animation as a video using ffmpeg writer
ani.save(video_filename, writer='ffmpeg', fps=10)

# Download the video file
files.download(video_filename)